# Door GUI and State Machine

This is the implementation of Door component. 

In [1]:
import paho.mqtt.client as mqtt
import logging
from threading import Thread
import json
from appJar import gui
from stmpy import Machine, Driver
import sqlite3 as sq
conn = sq.connect('Door.db')

In [2]:
##Initialize Database, only run once
c=conn.cursor()
c.execute('''CREATE TABLE rfids
             (rfid TEXT)''')

In [3]:
# MQTT broker address
MQTT_BROKER = 'mqtt.item.ntnu.no'
MQTT_PORT = 1883

# Topics for communication
MQTT_TOPIC_INPUT = 'ttm4115/team_1/door' # {"rfid":"1234"}
MQTT_TOPIC_OUTPUT = 'ttm4115/team_1/server' # {"command":push_door,"rfid":1234}

class DoorComponent:

    def on_connect(self, client, userdata, flags, rc):
        # we just log that we are connected
        self._logger.debug('MQTT connected to {}'.format(client))

    def on_message(self, client, userdata, msg):
        pass

    def __init__(self):
        # get the logger object for the component
        self._logger = logging.getLogger(__name__)
        print('logging under name {}.'.format(__name__))
        self._logger.info('Starting Component')

        # create a new MQTT client
        self._logger.debug('Connecting to MQTT broker {} at port {}'.format(MQTT_BROKER, MQTT_PORT))
        self.mqtt_client = mqtt.Client()
        # callback methods
        self.mqtt_client.on_connect = self.on_connect
        self.mqtt_client.on_message = self.on_message
        # Connect to the broker
        self.mqtt_client.connect(MQTT_BROKER, MQTT_PORT)
        # start the internal loop to process MQTT messages
        self.mqtt_client.loop_start()

        self.create_gui()
        
    def create_gui(self):
        self.app = gui("Door GUI")
        self.app.setStretch("both")
        self.app.setSticky("nesw")

        def publish_command(command):
            payload = json.dumps(command)
            self._logger.info(command)
            self.mqtt_client.publish(MQTT_TOPIC_INPUT, payload=payload, qos=2)
        
        self.app.startLabelFrame('Door Status')
        self.app.setStretch("both")
        self.app.setSticky("nesw")
        self.app.addLabel('title1', 'Locked')
        self.app.setLabelBg("title1", "red")
        self.app.addLabel('title2', 'Closed')
        self.app.setLabelBg("title2", "red")
        self.app.stopLabelFrame()

        self.app.startLabelFrame('Door Interactions')
        self.app.setStretch("both")
        self.app.setSticky("nesw")
        self.app.addLabelEntry("RFID")
        def on_button_pressed_proximity(button):
            rfidValue = self.app.getEntry("RFID") 
            if rfidValue == "1234":
                self.stm.send('rfid')
            else:
                self.app.infoBox("Wrong cat!", "An unknown RFID has been detected, the door will not open.", parent=None)
        self.app.addButton('A cat\'s RFID is detected', on_button_pressed_proximity)
        def on_button_pressed_push(button):
            self.stm.send('door_pushed')
        self.app.addButton('The cat pushes the door', on_button_pressed_push)
        def on_button_pressed_close(button):
            self.stm.send('door_closed')
            self.app.setLabel('title2', 'Closed')
            self.app.setLabelBg("title2", "red")
        self.app.addButton('The door closes', on_button_pressed_close)
        self.app.stopLabelFrame()
        def terminate():
            self.stm.terminate()
            return True
        self.app.setStopFunction(terminate)
    
    def unlock(self):
        self.app.setLabel('title1', 'Unlocked')
        self.app.setLabelBg("title1", "green")
    
    def door_opened(self):
        self.app.setLabel('title2', 'Opened')
        self.app.setLabelBg("title2", "green")
    
    def lock(self):
        self.app.setLabel('title1', 'Locked')
        self.app.setLabelBg("title1", "red")
        
 
t0 = {'source': 'initial', 'target': 'closed_locked'}
t1 = {'trigger': 'rfid', 'source': 'closed_locked', 'target': 'closed_unlocked',
      'effect': 'unlock; start_timer("t", "5000")'}
t2 = {'trigger': 'door_pushed', 'source': 'closed_unlocked', 'target': 'open_unlocked',
      'effect': 'stop_timer("t"); door_opened'}
t3 = {'trigger': 'door_closed', 'source': 'open_unlocked', 'target': 'closed_unlocked',
      'effect': 'start_timer("t", "5000")'}
t4 = {'trigger': 't', 'source': 'closed_unlocked', 'target': 'closed_locked',
      'effect': 'lock'}

s1 = {'name': 'closed_locked',
      'entry': 'lock'}
s2 = {'name': 'closed_unlocked'}
s3 = {'name': 'open_unlocked'}

# logging.DEBUG: Most fine-grained logging, printing everything
# logging.INFO:  Only the most important informational log items
# logging.WARN:  Show only warnings and errors.
# logging.ERROR: Show only error messages.
debug_level = logging.DEBUG
logger = logging.getLogger(__name__)
logger.setLevel(debug_level)
ch = logging.StreamHandler()
ch.setLevel(debug_level)
formatter = logging.Formatter('%(asctime)s - %(name)-12s - %(levelname)-8s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

d = DoorComponent()

stm_door = Machine(name='stm_door', transitions=[t0, t1, t2, t3, t4], obj=d, states=[s1, s2, s3])
d.stm = stm_door
driver = Driver()
driver.add_machine(stm_door)
driver.start()

d.app.go()

2020-04-16 17:05:10,062 - __main__     - INFO     - Starting Component
2020-04-16 17:05:10,065 - __main__     - DEBUG    - Connecting to MQTT broker mqtt.item.ntnu.no at port 1883
2020-04-16 17:05:10,073 - __main__     - DEBUG    - MQTT connected to <paho.mqtt.client.Client object at 0x000002CC9C9E1160>
2020-04-16 17:05:10,073 __main__:DEBUG MQTT connected to <paho.mqtt.client.Client object at 0x000002CC9C9E1160>
2020-04-16 17:05:10,208 appJar:WARNING [799/_checkMode]: Interactive mode is not fully tested, some features might not work.
2020-04-16 17:05:10,220 appJar:WARNING [810/_checkMode]: iPython is not fully tested, some features might not work.


logging under name __main__.
